# Part II: Modeling
---

[>>>Link to the presentation<<<](https://docs.google.com/presentation/d/1TooN8ycOvl-FB0gWikqjoO2LvPg2Vlh-09dW1wUhXB8/edit?usp=sharing)

In this section, the pre-processed datasets will be vectorized (converted into numbers) by a DictVectorizer so that computers will be able to understand tham
1. Data Preparation
2. Model Comparison
- K-mean
- LDA
- NMF
- Top2Vec > still in progress and will continue if time permits
- BERTopic > give up BERTopic because of time constraints

## 0. Import Libraries

In [2]:
# Import Basic Libraries
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# Import Sklearn Libraries
from sklearn.feature_extraction import DictVectorizer
from sklearn.decomposition import NMF
from sklearn.cluster import KMeans

# Import NLP Libraries
from gensim.models import LdaModel
from gensim.corpora.dictionary import Dictionary
from pythainlp.tokenize import sent_tokenize, word_tokenize
from pythainlp.corpus import thai_stopwords
import re
from tqdm import tqdm 
import pyLDAvis
import pyLDAvis.gensim_models
from top2vec import Top2Vec

# Set default Thai font
mpl.font_manager.fontManager.addfont('./THSarabunNew/THSarabunNew.ttf')
mpl.rc('font', family='TH Sarabun New', size=20)


# Set random seed
np.random.seed(42)

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

/Users/Anik/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


## 1. Data Preparation

### 1.1 Load Datasets

In [3]:
# Load datasets
bkkbiz = pd.read_json('../datasets/bkkbiz_digital_processed.json')
matichon = pd.read_json('../datasets/matichon_digital_processed.json')

In [ ]:
# Check shape
bkkbiz.shape

In [ ]:
# Check head
bkkbiz.head()

In [ ]:
# Check shape
matichon.shape

In [ ]:
# Check head
matichon.head()

### 1.2 Bag-of-Words: Vectorize with DictVectorizer

Since computer does not understand words as human does, they need to be converted into numbers by vectorizing. However, for Thai language, DictVectorizer will be used as a vectorizer.

In [4]:
# Function to store n_word in dict
def featurize(token_list):
    
    '''
    Description:
    Store word that occurs in the corpus in dictionary form
    
    Parameters:
    List of tokens
    
    Returns:
    Dictionary - token as key and occurance (one) as value
    
    '''
    
    token_list=token_list
    features = {}
    for token in token_list:
        features[token]=1
    return features

In [5]:
# Featurize words in corpus
bkkbiz_token = bkkbiz['article_tokenize'].apply(featurize)
matichon_token = matichon['article_tokenize'].apply(featurize)

In [6]:
# Vectorize words in corpus with DictVectorizer and store them as a sparse matrix
vectorizer_1 = DictVectorizer(sparse=True)
vectorizer_2 = DictVectorizer(sparse=True)

bkkbiz_vec = vectorizer_1.fit_transform(bkkbiz_token)
matichon_vec = vectorizer_2.fit_transform(matichon_token)

In [7]:
# Check origital number of articles
bkkbiz.shape, matichon.shape

((16464, 13), (10433, 10))

In [8]:
# Compare the new vectorized value
bkkbiz_vec, matichon_vec

(<16464x63496 sparse matrix of type '<class 'numpy.float64'>'
 	with 2760975 stored elements in Compressed Sparse Row format>,
 <10433x59082 sparse matrix of type '<class 'numpy.float64'>'
 	with 2135687 stored elements in Compressed Sparse Row format>)

## 2. Model Comparison

### 2.1 K-Means

In [ ]:
def kmeans(data_vec=None, n_clusters=10):
    '''
    Definition:
        - fit K-mean model
    Parameters:
        - vectorized data
        - number of clusters (number of topics)
    Returns:
        - fitted model
    '''
    
    kmeans = KMeans(n_clusters=n_clusters, 
                    init='k-means++', 
                    max_iter=100, 
                    n_init=1, 
                    random_state=42)

    model = kmeans.fit(data_vec)
    return model

In [ ]:
def kmeans_topic(model=None, n_clusters=10, corpus=None, vectorizer=None):
    '''
    Definition:
        - list out the topic and its related words
    Parameters:
        - model
        - number of clusters (number of topics)
    Retuens:
        - topics with top 30 related words
    '''
    order_centroids = model.cluster_centers_.argsort()[:, ::-1]
    terms = vectorizer.get_feature_names_out()
    print(corpus)
    print('K-Means')
    print('*'*20)
    for i in range(n_clusters):
        print(f'Topic {i+1}')
        print('-'*10)
        words = []
        for i in order_centroids[i,:30]:
            words.append(terms[i])
        print(words)
        print(' ')

In [ ]:
# Kmeans model result from Bkkbiz Corpus with 10 topics and 15 related words 
kmeans_bkkbiz = kmeans(data_vec=bkkbiz_vec, n_clusters=10)
kmeans_topic(model=kmeans_bkkbiz, n_clusters=10, corpus='BangkokBiz', vectorizer=vectorizer_1)

In [ ]:
# Kmeans model result from Matichon Corpus with 10 topics and 15 related words 
kmeans_matichon = kmeans(data_vec=matichon_vec, n_clusters=10)
kmeans_topic(model=kmeans_matichon, n_clusters=10, corpus='Matichon', vectorizer=vectorizer_2)

**Key Findings - KMean**
- KMeans match a text to a topic. One text belongs only to one topic, whereas a topic can have a multiple matching text.
- It seems that K-mean is successful at extracting topic and relevant keywords, however, labeling the topics is still not easy to perform, as there are many possibilies of choice for a K-mean topic. 

### 2.2 LDA

In [ ]:
# Function for modeling with LDA
def lda_model(data=None, num_topics=None): #, topicid=None
    
    '''
    Description:
    Fit documents to LDA model
    
    Parameters:
    - data (list)
    - number of topics
    
    Returns:
    - Perplexity rate
    - Topic and its top 15 associated words
    
    '''
    
    dictionary = Dictionary(data)
    corpus = [dictionary.doc2bow(txt) for txt in data]
    
    model = LdaModel(corpus=corpus, num_topics=num_topics)
    perplexity = model.log_perplexity(corpus)
    print(f'LDA Model with {num_topics} topics')
    print(f'Perplexity: {perplexity.round(2)}')
    print('-'*60)
    for i in range(num_topics):
        top_n = [dictionary[index] for index, prob in model.get_topic_terms(topicid=i, topn=30)]
        print(f'Topic {i+1}')
        print(top_n)
        print('-'*60)
    return model, corpus, dictionary

In [ ]:
# Visualization
#def lda_vis(data=None, num_topics=20):
def lda_vis(model=None, corpus=None, dictionary=None):    
    '''
    Description:
    Visualize LDA model
    
    Parameters:
    Fitted model, corpus and dictionary
    
    Returns:
    LDA visualisation
    
    '''
    
    #dictionary = Dictionary(data)
    #corpus = [dictionary.doc2bow(txt) for txt in data]
    #model = LdaModel(corpus=corpus, num_topics=num_topics)
    model=model
    pyLDAvis.enable_notebook()
    viz = pyLDAvis.gensim_models.prepare(model, corpus, dictionary)
    return viz

In [ ]:
# LDA model with BangkokBiz corpus
bkkbiz_lda_model, bkkbiz_lda_corpus, bkkbiz_lda_dictionary  = lda_model(data=bkkbiz['article_tokenize'], num_topics=10)

In [ ]:
# Visualization
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

lda_vis(model=bkkbiz_lda_model, corpus=bkkbiz_lda_corpus, dictionary=bkkbiz_lda_dictionary)

In [ ]:
# LDA model with matichon dataset
matichon_lda_model, matichon_lda_corpus, matichon_lda_dictionary = lda_model(data=matichon['article_tokenize'], num_topics=10)

In [ ]:
# Visualization
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

lda_vis(model=matichon_lda_model, corpus=matichon_lda_corpus, dictionary=matichon_lda_dictionary)

**Key Findings - LDA**
- LDA is a popular algorithm for topic-modeling. It extracts topics from documents, where one documents might show multiple topics. This is the point where LDA differentiate itself from KMeans. 
- However, one of the biggest challenges of LDA is that users have to define the number of topic by him/herself. 
- Considering the number of topics might be unknown, we might missed some insight from tuning the false number of topics.
- The visualization with pyLDAvis helps a lot to identify the ideal number of topics, where the more overlaps, the more we should consider reducing the number of topics. 
- However, we can see form both visualization that even with 10 topics, there are sill overlaps and the topics are at the same time not successful in filtering the articles related to Digital Competitiveness Ranking.

### 2.3 NMF

In [ ]:
# Create function for modeling with NMF
def nmf(data=None, n_components=10, vectorizer=None):
    
    '''
    Description:
    Fit documents to NMF model
    
    Parameters:
    - Data (vectorized)
    - Number of components (number of topics)
    - Vectorizer
    
    Returns:
    Fitted model
    
    '''
    
    model = NMF(n_components=n_components)
    model.fit(data)
    features = model.transform(data)
    components = pd.DataFrame(model.components_,
                                columns=vectorizer.get_feature_names())
    # Topic & Words
    
    for i in range(n_components):
        words = []
        topic = components.iloc[i]
        print(f'Top 15 words for topic {i+1} are:')
        for n, word in enumerate(topic.nlargest(30)):
            words.append(topic.nlargest(30).keys()[n])
        print(words)
        print('\n')
    return model

In [ ]:
bkkbiz_nmf_model = nmf(data=bkkbiz_vec, n_components=10, vectorizer=vec_bkkbiz)

In [ ]:
matichon_nmf_model = nmf(data=matichon_vec, n_components=10, vectorizer=vec_matichon)

**Key Findings**
- NMF model calculates topic x terms x documents together. 
- However, it shows the same problem as LDA that it is not easy to lable a topic with a title.

### 2.4 Top2Vec

In [9]:
# Create function that tokenize, remove stopwords and perform RegularExpression
def tokenize(txt):
  
    ''' 
    Description:
        A function that returns tokenized words with stop words and other regular expressions 
        that do not match English and Thai alphabets removed

    Parameters:
        txt = text string

    Return:
        word tokens as list
    '''

    stopwords = thai_stopwords()
    tokens = word_tokenize(txt, keep_whitespace = False)
    tokens = [token for token in tokens if token not in stopwords]
    tokens = [token for token in tokens if re.match(r'[A-Za-zก-\uDe5c]+', token)]
    return tokens

In [10]:
# Create function for modeling with Top2Vec
def top2vec(documents=None, speed='fast-learn', ngram_vocab=None):
    
    '''
    Description:
        - fit Top2Vec model
    Parameters:
        - documents(in list format)
        - number of topics
        - learning speed: fast-learn, learn or deep-learn
    Return:
        - model
        - topic numbers
        - topic words
    '''
    
    model = Top2Vec(documents=documents,
                      speed=speed,
                      tokenizer=tokenize)
    topic_words, word_scores, topic_nums = model.get_topics()
    print(f'Topic: {topic_nums+1}')
    print(f'Words: {topic_words}')
    return model, topic_nums, topic_words

In [ ]:
def top2vec_doc(model=None, topic_num=None):
    documents, document_scores, document_ids = \
    model.search_documents_by_topic(topic_num=topic_num, num_docs=1) # Reduce to 1 to save space on GitHub
    for doc, score, doc_id in zip(documents, document_scores, document_ids):
        print(f'Document: {doc_id}, Score: {score}')
        print('-'*20)
        print(doc)
        print('-'*20)
        print()

In [12]:
# Prepare input data as list
bkkbiz_list = bkkbiz['article'].tolist()
matichon_list = matichon['article'].tolist()

**Reference for variable names**

|number|speed|
|---|---|
|bX|Bkkbiz|
|mX|Matichon|
|X1|fast-learn|
|X2|learn|
|X3|deep-learn|


e.g. 
- model_b1 = Top2Vec model with fast-learn speed fitted on BangkokBiz corpus
- model_m3 = Top2Vec model with deep-learn speed fitted on Matichon corpus

**2.4.1 Fast-learn model**

In [14]:
# Bkkbiz Fast-learn model
model_b1 = Top2Vec(documents=bkkbiz_list,speed='fast-learn',tokenizer=tokenize)
b1_topic_words,b1_word_scores, b1_topic_nums = model_b1.get_topics()

2022-11-21 01:52:28,468 - top2vec - INFO - Pre-processing documents for training
2022-11-21 01:54:37,132 - top2vec - INFO - Creating joint document/word embedding
2022-11-21 02:04:11,588 - top2vec - INFO - Creating lower dimension embedding of documents
2022-11-21 02:04:28,862 - top2vec - INFO - Finding dense areas of documents
2022-11-21 02:04:31,784 - top2vec - INFO - Finding topics


In [15]:
# Calling topic and related words from BangkokBiz corpus from fast-learn model
b1_topic_words[0:20] 

array([['ธนาคาร', 'สินเชื่อ', 'งกิ้ง', 'ธนาคารพาณิชย์', 'แบงก์',
        'ทางการเงิน', 'แบงก์กิ้ง', 'กสิกร', 'Banking', 'SCB',
        'ค่าธรรมเนียม', 'NPL', 'NIM', 'กรุงศรี', 'สถาบันการเงิน',
        'เงินฝาก', 'Lending', 'ลูกค้า', 'ธนาคารแห่งประเทศไทย',
        'ดอกเบี้ย', 'ธุรกรรม', 'แบ', 'กรุงศรีอยุธยา', 'เอ็นพีแอล',
        'หนี้สูญ', 'EASY', 'Financial', 'หนี้', 'Krungsri', 'ttb',
        'การเงิน', 'KBANK', 'อัตราดอกเบี้ย', 'SME', 'เงินสด',
        'สินทรัพย์', 'หนี้เสีย', 'ลูกหนี้', 'พาณิชย์', 'ชำระเงิน',
        'บัญชีเงินฝาก', 'สิ้นปี', 'เปิดบัญชี', 'เงินได้', 'Bank', 'ธปท.',
        'บัตรเครดิต', 'กรุงไทย', 'เชื่อได้', 'ผยง'],
       ['องค์กร', 'ฟอ', 'ทรานส์', 'ไอที', 'การเปลี่ยนแปลง', 'เทคโนโลยี',
        'คอมพิวติ้ง', 'เอ', 'ไอ', 'สฟอร์', 'ออราเคิล', 'การทำงาน', 'ร์',
        'ค็อก', 'ไฮบริด', 'โซลูชั่น', 'ทิฟ', '์ท', 'ชั่น', 'เคียว',
        'ปรับเปลี่ยน', 'ดจ์', 'ไอบีเอ็ม', 'คลาวด์', 'ความยืดหยุ่น',
        'การประมวลผล', 'ยืดหยุ่น', 'ปัญญาประดิษฐ์', 'ธิงส์(', 'ธุรกิจ',


In [16]:
# Call topic related documents
top2vec_doc(model=model_b1, topic_num=1)

Document: 16138, Score: 0.8035717010498047
--------------------
นายนพดล ปัญญาธิปัตย์ กรรมการผู้จัดการประจำประเทศไทย เดลล์ เทคโนโลยีส์ กล่าวว่า โลกธุรกิจปัจจุบันมีความซับซ้อนเพิ่มมากขึ้น ต้องเผชิญกับปัจจัยโรคระบาด จำเป็นต้องมองหาสิ่งใหม่ๆ ที่จะทำให้เกิดโอกาสการเติบโต


ดังนั้นระบบไอทีต้องมีการยกระดับสู่รูปแบบอัตโนมัติ ให้ความสำคัญกับไซเบอร์ซิเคียวริตี้ ขณะเดียวกันมีความสามารถที่จะบริหารจัดการมัลติคลาวด์

เขากล่าวว่า องค์กรเริ่มหันมาให้ความสำคัญกับการบริหารจัดการข้อมูลมากขึ้น โดยที่เริ่มดำเนินการแล้วเช่น ธุรกิจด้านการเงินการธนาคาร โทรคมนาคม และต่อไปที่จะแพร่หลายมากขึ้นคาดว่าจะเป็นกลุ่มอีคอมเมิร์ซและดิลิเวอรีเพื่อการเข้าถึงความต้องการของลูกค้าได้ดีมากขึ้น รวมถึงรองรับบริหารจัดการฐานลูกค้าที่มีมากขึ้น

"เราได้เห็นองค์กรพลิกโฉมระบบไอทีเพื่อรองรับการเปลี่ยนแปลงหรือดิสรัปชันต่างๆ ดังที่จะเห็นได้จากช่วงสองสามปีที่ผ่านมาที่องค์กรได้เริ่มวางแผนและลงมือทำดิจิทัลทรานส์ฟอร์เมชันอย่างเป็นรูปธรรม"
--------------------

Document: 13567, Score: 0.7443233132362366
--------------------
ทุกวันนี้ องค์กรต่

In [21]:
# Matichon Fast-learn model
model_m1 = Top2Vec(documents=matichon_list,speed='fast-learn',tokenizer=tokenize)
m1_topic_words,m1_word_scores, m1_topic_nums = model_m1.get_topics()

2022-11-21 02:08:51,492 - top2vec - INFO - Pre-processing documents for training
2022-11-21 02:10:07,019 - top2vec - INFO - Creating joint document/word embedding
2022-11-21 02:15:44,531 - top2vec - INFO - Creating lower dimension embedding of documents
2022-11-21 02:15:48,280 - top2vec - INFO - Finding dense areas of documents
2022-11-21 02:15:48,548 - top2vec - INFO - Finding topics


In [22]:
# Calling topic and related words from BangkokBiz corpus from fast-learn model
m1_topic_words[0:20]

array([['ดำเนินคดี', 'กระทำความผิด', 'ความผิด', 'เข้าข่าย', 'แจ้งความ',
        'คำสั่งศาล', 'ปราบปราม', 'พนักงานสอบสวน', 'ผิดกฎหมาย', 'จับกุม',
        'โพสต์', 'การกระทำ', 'บังคับการ', 'พ.ต.อ.', 'ท.)', 'การจับกุม',
        'พยานหลักฐาน', 'ปอ', 'เจ้าหน้าที่ตำรวจ', 'ปลอม', 'ตามกฎหมาย',
        'เท็จ', 'สำนักงานตำรวจแห่งชาติ', 'บิดเบือน', 'มีความผิด',
        'ผู้ต้องหา', 'ผู้เสียหาย', 'ท.', 'ผู้กระทำผิด', 'ปิดกั้น',
        'การปิดกั้น', 'ศาล', 'ประมวลกฎหมายอาญา', 'หลอกลวง', 'เพจ',
        'หมายศาล', 'ข้อความ', 'การสืบสวน', 'เอาผิด', 'อาชญากรรม',
        'ข้อหา', 'สืบสวน', 'หมายจับ', 'ตำรวจ', 'ระงับ', 'พล.ต.ต.', 'บก',
        'เฟซบุ๊ก', 'มีโทษ', 'พุทธิ'],
       ['ฟื้นตัว', 'จีดีพี', 'หนี้', 'ขยายตัว', 'เศรษฐกิจ', 'ชะลอตัว',
        'ภาวะเศรษฐกิจ', 'ค่าเงินบาท', 'การส่งออก', 'แข็งค่า', 'เงินเฟ้อ',
        'ตลาดหุ้น', 'สภาพคล่อง', 'การฟื้นตัว', 'การลงทุน', 'ติดลบ',
        'หดตัว', 'การคลัง', 'ความผันผวน', 'ผลกระทบ', 'นักลงทุน',
        'ความไม่แน่นอน', 'ผันผวน', 'ระบบเศรษฐกิจ', 'คาด', 

In [23]:
# Call topic related documents
top2vec_doc(model=model_m1, topic_num=1)

Document: 3172, Score: 0.7673107385635376
--------------------
นายรพี สุจริตกุล เลขาธิการสำนักงานคณะกรรมการกำกับหลักทรัพย์และตลาดหลักทรัพย์(ก.ล.ต.) กล่าวว่า สถานการณ์เงินทุนไหลออกในตลาดหุ้นช่วงนี้มองว่าเป็นผลกระทบระยะสั้น แต่ท้ายที่สุดแล้วก็จะต้องดูพื้นฐานของบริษัทหรือหุ้นที่ลงทุนมากกว่า แนวโน้มการดำเนินงานเป็นอย่างไรและบริษัทมีทิศทางการดำเนินงานอย่างไร เพราะหากพื้นฐานดีย่อมมีเม็ดเงินเข้ามาลงทุน ซึ่งเรื่องที่สำคัญคือการปรับตัวรองรับกับโลกดิจิทัลที่จะเข้ามา เพราะธุรกิจเดิมจะถูกกระทบและทำให้เกิดธุรกิจใหม่ หากบริษัทไม่ปรับตัวก็จะไม่สามารถอยู่รอดได้
" ขณะนี้ไม่เห็นแนวโน้มว่าจะเกิดความเสียหายหรือเกิดวิกฤติเศรษฐกิจซ้ำรอยอีก เพราะครั้งนี้นเกิดจากสถาบันการเงินที่มีการปล่อยกู้และมีหนี้เสียจำนวนมาก แต่ปัจจุบันสถาบันการเงินแข็งแกร่ง เงินกองทุนสูง และหนี้เสียอยู่ในระดับต่ำ อย่างไรก็ดีมีปัจจัยที่กังวล แต่ไม่ใช่ธุรกิจการเงินเท่านั้น แต่คือทุกธุรกิจที่จะต้องมีการปรับตัวรับเทคโนโลยีที่เปลี่ยนแปลงไป เพราะหากไม่ปรับตัวก็จะมีธุรกิจใหม่เข้ามาแทนที่ล้มหายตายจากไป" นายรพี กล่าว
--------------------

Documen

**2.4.2 Model with speed=learn**

In [24]:
# Bkkbiz 'learn' model
model_b2 = Top2Vec(documents=bkkbiz_list,speed='learn',tokenizer=tokenize)
b2_topic_words,b2_word_scores, b2_topic_nums = model_b2.get_topics()

2022-11-21 02:15:48,873 - top2vec - INFO - Pre-processing documents for training
2022-11-21 02:17:58,941 - top2vec - INFO - Creating joint document/word embedding
2022-11-21 02:33:19,781 - top2vec - INFO - Creating lower dimension embedding of documents
2022-11-21 02:33:36,815 - top2vec - INFO - Finding dense areas of documents
2022-11-21 02:33:41,013 - top2vec - INFO - Finding topics


In [25]:
# Calling topic and related words from BangkokBiz corpus from 'learn' model
b2_topic_words[0:20]

array([['องค์กร', 'ไอที', 'คอมพิวติ้ง', 'ไอ', 'ทรานส์', 'ซับซ้อน',
        'คลาวด์', 'ความยืดหยุ่น', '์ท', 'ฟอ', 'ออราเคิล', 'เอ', 'ค็อก',
        'การทำงาน', 'ดจ์', 'ยืดหยุ่น', 'ไฮบริด', 'ปัญญาประดิษฐ์', 'ทิฟ',
        'ไอบีเอ็ม', 'เลิร์น', 'โซลูชั่น', 'การประมวลผล', 'ธิงส์(',
        'เทคโนโลยี', 'สภาพแวดล้อม', 'แมชชีน', 'ร์', 'ธิงส์',
        'การเปลี่ยนแปลง', 'โอที', 'ปรับเปลี่ยน', 'ควอนตัม', 'ไล',
        'เคียว', 'สฟอร์', 'จำเป็นต้อง', 'ปฐ', 'คล่องตัว', 'นิง',
        'แปซิฟิก', 'ความคาดหวัง', 'เข้ากับ', 'กุญแจ', 'ผลลัพธ์',
        'พนักงาน', 'ชั่น', 'ความสามารถ', 'เทรนด์', 'ตื่นตัว'],
       ['น์', 'บิท', 'บิตคอยน์', 'สกุล', 'คอย', 'คริปโทเคอร์เรนซี',
        'ดิ่ง', 'คริปโตเคอร์เรนซี', 'เรียม', 'สลา', 'ธนาคารกลาง', 'แด็ก',
        'มัสก์', 'ตามเวลา', 'ร่วง', 'ทะยาน', 'ดอลลาร์', 'น์"', 'ฟองสบู่',
        'ซื้อขาย', 'Bitcoin', 'ป', 'สหรัฐ', 'ลอน', 'การซื้อขาย',
        'สำนักข่าว', 'ทวีต', 'บรา', 'เหมือง', 'ทรุดตัว', 'สั่งห้าม',
        'การเก็งกำไร', 'ซื้อขายแลกเปลี่ยน', 'เหรียญ

In [26]:
# Call topic related documents
top2vec_doc(model=model_b2, topic_num=1)

Document: 14502, Score: 0.7961863875389099
--------------------
การปรับตัวลงของบิตคอยน์ในช่วงเช้าวันนี้ เป็นการร่วงลงต่อเนื่องหลังจากเมื่อวันพฤหัสบดี(9ธ.ค.)บิตคอยน์ร่วงลงทะลุระดับ 50,000 ดอลลาร์ ท่ามกลางความกังวลเกี่ยวกับการที่ทางการสหรัฐจะเข้าควบคุมการซื้อขายสกุลเงินดิจิทัล

เมื่อเวลา 19.25 น.ตามเวลาไทย บิตคอยน์ดิ่งลง 2.73% สู่ระดับ 49,383.89 ดอลลาร์ในการซื้อขายบนแพลตฟอร์ม Coinbase

นักลงทุนกังวลว่า สหรัฐจะคุมเข้มการทำธุรกรรมสกุลเงินคริปโทฯ หลังจากที่สภาคองเกรสเรียกผู้บริหารของแพลตฟอร์มซื้อขายสกุลเงินดิจิทัลรายใหญ่ รวมทั้งผู้ที่เกี่ยวข้องกับธุรกิจขุดเหมืองบิตคอยน์เข้าให้การต่อคณะกรรมาธิการบริการการเงินประจำสภาผู้แทนราษฎรเมื่อวานนี้

ขณะนี้ทางการสหรัฐกำลังพิจารณาเกี่ยวกับการควบคุมธุรกรรมสกุลเงินดิจิทัล โดยอาจให้อยู่ภายใต้การกำกับดูแลของสำนักงานคณะกรรมการกำกับหลักทรัพย์และตลาดหลักทรัพย์ของสหรัฐ (กลต.) หรืออาจมีการตั้งหน่วยงานใหม่ขึ้นมากำหนดกฎระเบียบควบคุมธุรกิจดังกล่าว

นอกจากนี้ บิตคอยน์ยังถูกกดดันจากความกังวลที่ว่าสหรัฐอาจเรียกเก็บภาษีสกุลเงินคริปโทฯ

ทั้งนี้ นักลงทุนวิตกว่าสหรัฐจะทำก

In [27]:
# Matichon 'learn' model
model_m2 = Top2Vec(documents=matichon_list,speed='learn',tokenizer=tokenize)
m2_topic_words,m2_word_scores, m2_topic_nums = model_m2.get_topics()

2022-11-21 02:33:42,478 - top2vec - INFO - Pre-processing documents for training
2022-11-21 02:35:37,268 - top2vec - INFO - Creating joint document/word embedding
2022-11-21 02:46:23,487 - top2vec - INFO - Creating lower dimension embedding of documents
2022-11-21 02:46:28,033 - top2vec - INFO - Finding dense areas of documents
2022-11-21 02:46:28,335 - top2vec - INFO - Finding topics


In [28]:
# Calling topic and related words from BangkokBiz corpus from 'learn' model
m2_topic_words[0:20]

array([['ความผิด', 'ดำเนินคดี', 'คำสั่งศาล', 'กระทำความผิด', 'บิดเบือน',
        'การกระทำ', 'มีความผิด', 'การปิดกั้น', 'เข้าข่าย',
        'ประมวลกฎหมายอาญา', 'ปิดกั้น', 'โพสต์', 'มีโทษ', 'ผิดกฎหมาย',
        'ปลุกปั่น', 'เอาผิด', 'เพจ', 'ระงับ', 'หมิ่น', 'เท็จ',
        'ผู้กระทำผิด', 'ฝ่าฝืน', 'ลบ', 'แจ้งความ', 'หมายศาล',
        'ตามกฎหมาย', 'อาญา', 'ตื่นตระหนก', 'ศาล', 'ปลอม', 'พยานหลักฐาน',
        'ท.', 'ข้อความ', 'จำคุก', 'ข้อกฎหมาย', 'ความสงบเรียบร้อย',
        'เจตนา', 'หมิ่นประมาท', 'คอมพิวเตอร์', 'ทำผิด', 'ทวิตเตอร์',
        'ทางกฎหมาย', 'เจ้าหน้าที่ตำรวจ', 'กระทำการ', 'ความร้ายแรง',
        'การจับกุม', 'เบาะแส', 'มาตรา', 'เฟค', 'ฟ้อง'],
       ['คลาวด์', 'องค์กร', 'Cloud', 'Edge', 'โซลูชั่น', 'SAP',
        'การทำงาน', 'โซลูชัน', 'ไอที', 'ยืดหยุ่น', 'ซับซ้อน', 'Aruba',
        'ทรานส์', 'ชาญฉลาด', 'ความยืดหยุ่น', 'Process', 'สภาพแวดล้อม',
        'เอบี', 'ซอฟต์แวร์', 'คล่องตัว', 'Enterprise', 'ERP', 'RPA',
        'คอมพิวติ้ง', 'ฮาร์ดแวร์', 'NTT', 'Analytics', 'การประมวล

In [29]:
# Call topic related documents
top2vec_doc(model=model_m2, topic_num=1)

Document: 10098, Score: 0.6015714406967163
--------------------
การเปลี่ยนแปลงที่เกิดขึ้น ไม่ว่าจะเป็นนโยบายทำงานจากที่บ้าน ไปจนถึงความต้องการของลูกค้าที่เพิ่มขึ้นทั้งด้านการบริการและต้องการประสบการณ์ที่ดีขึ้น ทำให้องค์กรต่างๆ จำเป็นที่จะต้องปรับตัวให้เร็วขึ้น เพื่อรับมือกับผลกระทบจากการแพร่ระบาดของโควิด-19 เรดแฮทคาดว่าแนวโน้มทางเทคโนโลยีจะยังคงมีการพัฒนาอย่างต่อเนื่อง เพราะการพึ่งพาเทคโนโลยีทวีความสำคัญมากขึ้นในสภาพสังคมและเศรษฐกิจในปัจจุบัน องค์กรในภูมิภาคเอเชียแปซิฟิก (APAC) ที่ต้องการอยู่ให้รอดจากสถานการณ์เช่นนี้ควรทบทวนกลยุทธ์ด้านดิจิทัลทรานส์ฟอร์เมชั่น โดยอ้างอิงจากแนวโน้มที่เร้ดแฮทคาดว่าจะเกิดขึ้นในปีนี้
5G, IoT และ Edge computing คือ สามเทคโนโลยีแห่งการเชื่อมต่อที่ชาญฉลาด
ภูมิภาคเอเชียแปซิฟิกจะพัฒนาเส้นทาง 5G ของตนอย่างต่อเนื่อง ในขณะที่บริการ 5G เชิงพาณิชย์ได้เปิดให้บริการแล้วในเก้าประเทศ รวมถึงเกาหลีใต้ ญี่ปุ่น และจีน และยังมีอีก 12 ประเทศที่ได้ประกาศแผนการทำนองเดียวกันนี้อย่างเป็นทางการ ความพร้อมในการใช้งาน 5G ที่เพิ่มขึ้นจะช่วยขับเคลื่อนอินเทอร์เน็ตออฟธิงค์ (IoT) และเอดจ์คอ

**2.4.3 Deep-learn model**

In [ ]:
# Bkkbiz 'deep-learn' model
model_b3 = Top2Vec(documents=bkkbiz_list, speed='deep-learn', tokenizer=tokenize)
b3_topic_words, b3_word_scores, b3_topic_nums = model_b3.get_topics()

2022-11-21 03:25:46,856 - top2vec - INFO - Pre-processing documents for training
2022-11-21 03:28:00,825 - top2vec - INFO - Creating joint document/word embedding


In [ ]:
# Calling topic and related words from BangkokBiz corpus from 'learn' model
b3_topic_words[0:5]

In [ ]:
# Call topic related documents
top2vec_doc(model=model_b3, topic_num=1)

In [ ]:
# Matichon 'deep-learn' model
model_m3 = Top2Vec(documents=matichon_list, speed='learn', tokenizer=tokenize)
m3_topic_words, m3_word_scores, m3_topic_nums = model_m3.get_topics()

In [ ]:
# Calling topic and related words from BangkokBiz corpus from 'learn' model
m3_topic_words[0:20]

In [ ]:
# Call topic related documents
top2vec_doc(model=model_m3, topic_num=1)

# Part 3: Conclusion and Next Steps

**Conclusion**
- The Top2Vec models perform the best among 4 available choices.
- With Top2Vec, we are successfully filter related articles on the topics regarding Digital Competitiveness Ranking.
- We successfully map back the topics to each soft-data based indicators as follows:

|Domain|Indicator| Top2Vec | Topic |
|---|---|---|---|
|Knowledge| > Foreign highly-skilled personnel| | Education |
|  |> Digital/Technological skills | |Digital skills|
| |> Employee Training| 
| |> Knowledge Transfer|
| Law&Regulation| > Immigration Law|| Digital skills| 
|| > Scientific research legislation||Smart city
|| > Intellectual property rights| |Law and Crime|
|| > Cyber security| | AI|
|| > Development and Application of Technology||EEC
|Investment| > Venture capital| | Venture capital|
||> Funding of technological development||
||>Public-private partnerships
|Infrastructure|> Management of cities||Smart city|
||> Banking and financial services||Banking
||> Communication technology||Telecommunications
|Digital Economy|> Agility of companies||Venture capital
||> Opportunities and threats||AI
||> Use of big data and analytics||EEC
||> Internet experience

- These can be used as text filter for further policy analysis.
- This can help scan through a vast number of texts and thus saves time and increase higher text accuracy.

**Next-Steps**
- Try BERTopic model, where it helps find the most appropriate topic and documents by maximizing candidate relevance.
- This model requires a heavy research to understand how it was trained, how it performs and how can the parameters be tuned to fit with Thai language.

**Potential Use Case at True**
- Topic modeling with True's after-sales services data from call-center, non-voice services, IVR and Mari
- Potential impact:
    - Categorizing after-sales data in a more granular level
    - Reduce human errors during data input stage
    - Finding new-emerging after-sales service clusters